# Create DB for all triples

In [43]:
import pandas as pd
from tqdm.notebook import tqdm
import os
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Download/main.db')

In [22]:
PATH = "Download/Cleaned Data/"

for file in os.listdir(PATH):
    
    df = pd.read_csv(os.path.join(PATH,file))

    df.to_sql(os.path.splitext(file)[0], con=engine, if_exists='replace')

In [24]:
engine.table_names()

['Confession',
 'Confessions',
 'CreepyPasta',
 'NoSleep',
 'Self',
 'ShortScaryStories',
 'SpanishText',
 'StoriesAboutKevin',
 'TIFU']

In [27]:
df = pd.read_sql_table('NoSleep', 'sqlite:///Download/main.db')  
# df['selftext'].to_list()
df

,index,created_utc,id,score,selftext,subreddit,title,title_language,selftext_language
0,0,1269397810,bhef5,24,Hey guys! Thanks for checking out /r/NoSleep! ...,nosleep,Welcome to /r/NoSleep!,nl,en
1,1,1269458850,bhq4w,68,This is copypasta from an earlier post... no w...,nosleep,The only possible supernatural experience I've...,en,en
2,2,1269476233,bhtlo,19,Reposting at the request of another. This was ...,nosleep,The Longhorn,en,en
3,3,1269479929,bhu80,29,"So, after looking at some other subreddits, I ...",nosleep,/r/NoSleep Rules,af,en
4,4,1269481302,bhufy,18,This is copy and pasted from the thread that s...,nosleep,My dad's story,cy,en
...,...,...,...,...,...,...,...,...,...
173274,173274,1591452441,gxrm8v,1,"""Any sign of 'em yet?"" \n\nI continued staring...",nosleep,I'm tasked with killing nameless things out in...,en,en
173275,173275,1591452568,gxrnj7,1,They say the devil is in the details. Well th...,nosleep,"I picked up a hitchhiker by mistake, now he's ...",en,en
173276,173276,1591453707,gxrytp,1,It is hard for me to talk about my old friend ...,nosleep,My Best Friend Saw Bugs Under His Skin,de,en
173277,173277,1591454493,gxs6jf,1,This is the only rule of our household. If you...,nosleep,Do NOT open your eyes. (The Beginning),en,en


In [30]:
import spacy
nlp = spacy.load('en_core_web_lg')

from openie import StanfordOpenIE
import truecase
from tqdm.notebook import tqdm

with StanfordOpenIE() as client:
    for table in tqdm(engine.table_names()):
        # here i only have first 1000 rows per subreddit since the processing time is so slow
        df = pd.read_sql_table(table, 'sqlite:///Download/main.db').head(1000)
        df_li = []
        for story_id, story in tqdm(zip(df['id'].to_list(), df['selftext'].to_list()), total = len(df['id'].to_list())):
            doc = nlp(story)
            for sent in doc.sents:
                original_text = sent.text
                truecase_text = truecase.get_true_case(original_text)
                text = truecase_text[0].lower() + truecase_text[1:]
                for triple in client.annotate(text):
                    triple['id'] = story_id
                    triple['subreddit'] = table
                    triple['original sent'] = text
                    df_li.append(triple)
            
        kg_df = pd.DataFrame(df_li)
        kg_df.to_sql(f"{table}_kg", con=engine, if_exists='replace')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Starting server with command: java -Xmx8G -cp /home/anthony/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-b7156e21bcdc44a4.props -preload openie


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [2]:
engine.table_names()

['Confession',
 'Confession_kg',
 'Confessions',
 'Confessions_kg',
 'CreepyPasta',
 'CreepyPasta_kg',
 'NoSleep',
 'NoSleep_kg',
 'Self',
 'Self_kg',
 'ShortScaryStories',
 'ShortScaryStories_kg',
 'SpanishText',
 'SpanishText_kg',
 'StoriesAboutKevin',
 'StoriesAboutKevin_kg',
 'TIFU',
 'TIFU_kg']

In [3]:
pd.DataFrame(engine.execute("SELECT * FROM NoSleep_kg LIMIT 5").fetchall())

,0,1,2,3,4,5,6
0,0,i,would really like,see,bhef5,NoSleep,i would really like to see this get off the gr...
1,1,i,would like,see,bhef5,NoSleep,i would really like to see this get off the gr...
2,2,I,love,scared,bhef5,NoSleep,* I love being scared
3,3,I,being,scared,bhef5,NoSleep,* I love being scared
4,4,you,being,scared,bhef5,NoSleep,* you love being scared


# Generate knowledge graph and support queries using Neo4j¶

In [2]:
from py2neo import Graph
from py2neo import Node, Relationship

In [56]:
CREEPY_SUBREDDIT = ['CreepyPasta_kg','NoSleep_kg','ShortScaryStories_kg']
NONCREEPY_SUBREDDIT = ['Confession_kg','Confessions_kg','Self_kg','StoriesAboutKevin_kg','TIFU_kg']
IGNORED_SUBREDDIT = ['SpanishText_kg']

df_li=[]
for dfname in CREEPY_SUBREDDIT:
    df = pd.DataFrame(engine.execute(f"SELECT * FROM {dfname}").fetchall(),
                     columns=['index','subject','relation','object','id','subreddit','originial_sent'])
    df_li.append(df)
    
creepy_kg_df = pd.concat(df_li)
creepy_kg_df = creepy_kg_df.applymap(str)
creepy_kg_df.to_sql(f"creepy_kg", con=engine, if_exists='replace')

In [57]:
CREEPY_SUBREDDIT = ['CreepyPasta_kg','NoSleep_kg','ShortScaryStories_kg']
NONCREEPY_SUBREDDIT = ['Confession_kg','Confessions_kg','Self_kg','StoriesAboutKevin_kg','TIFU_kg']
IGNORED_SUBREDDIT = ['SpanishText_kg']

df_li=[]
for dfname in NONCREEPY_SUBREDDIT:
    df = pd.DataFrame(engine.execute(f"SELECT * FROM {dfname}").fetchall(),
                     columns=['index','subject','relation','object','id','subreddit','originial_sent'])
    df_li.append(df)
    
noncreepy_kg_df = pd.concat(df_li)
noncreepy_kg_df = noncreepy_kg_df.applymap(str)
noncreepy_kg_df.to_sql(f"noncreepy_kg", con=engine, if_exists='replace')

In [48]:
engine.table_names()

['Confession',
 'Confession_kg',
 'Confessions',
 'Confessions_kg',
 'CreepyPasta',
 'CreepyPasta_kg',
 'NoSleep',
 'NoSleep_kg',
 'Self',
 'Self_kg',
 'ShortScaryStories',
 'ShortScaryStories_kg',
 'SpanishText',
 'SpanishText_kg',
 'StoriesAboutKevin',
 'StoriesAboutKevin_kg',
 'TIFU',
 'TIFU_kg',
 'creepy_kg',
 'noncreepy_kg']

In [27]:
from tqdm.notebook import tqdm
# Connect to Neo4j using corresponding <port:7687> and <password>
graph = Graph("bolt://localhost:11003", auth = ("neo4j", "Neo4j"))
# graph.delete_all()

In [60]:
creepy_kg_df

,index,subject,relation,object,id,subreddit,originial_sent
0,0,i,was,sure,g7y03,CreepyPasta,"i was sure there was one involving Candlejack,..."
1,1,one,involving,Candlejack,g7y03,CreepyPasta,"i was sure there was one involving Candlejack,..."
2,2,eddie,had,one sister,gn6u7,CreepyPasta,eddie had one sister.
3,3,he,walked at_time,one day,gn6u7,CreepyPasta,one day he walked into the bathroom
4,4,he,walked into,bathroom,gn6u7,CreepyPasta,one day he walked into the bathroom
...,...,...,...,...,...,...,...
27976,27976,it,has,hands,z9wq6,ShortScaryStories,"oh God, it's Fucking hands."
27977,27977,it,has,Fucking hands,z9wq6,ShortScaryStories,"oh God, it's Fucking hands."
27978,27978,it,'s moves in,in kill,z9wq6,ShortScaryStories,"it's moves in for the kill, you pinch yourself..."
27979,27979,it,has,moves,z9wq6,ShortScaryStories,"it's moves in for the kill, you pinch yourself..."


In [61]:
entities = {}

# Parse the entities and build the knowledge graph in Neo4j Database
for index, row in tqdm(creepy_kg_df.iterrows(), total=len(creepy_kg_df)):
    index, sub, rel, obj, story_id, subreddit, original_sent = row
    
    sub_node = graph.nodes.match(entities.get(sub, "Object"), name=sub).first()
    obj_node = graph.nodes.match("Object", name=obj).first()
    if not sub_node:
        sub_node = Node(entities.get(sub, "Object"),name = sub)
    if not obj_node:
        obj_node = Node(entities.get(obj, "Object"),name = obj)
    relation = Relationship.type(rel)    
    graph.merge(relation(sub_node, obj_node), entities.get(sub, "Object"), "name" )

  0%|          | 0/272034 [00:00<?, ?it/s]

In [20]:
graph.call.dbms.components()

 name         | versions  | edition    
--------------|-----------|------------
 Neo4j Kernel | ['4.2.2'] | enterprise 

In [29]:
def describe(graph, node):
    nodes = graph.run("MATCH ({name: '%s'})-[r]->(m)RETURN r,m" % node)
    flag=True
    for node in nodes:
        #print(node['m']['name'])
        print(node['r'])
        flag = False
    if flag:
        print("{} not found".format(actor))

In [31]:
describe(graph,'him')

(him)-[:go {}]->(I stared)
(him)-[:go {}]->(I stared at mirror)
(him)-[:have laid up {}]->(there)
(him)-[:have pulled off {}]->(it)
(him)-[:start {}]->(chuckle)
(him)-[:start {}]->(giggling)


# networkx answer questions

In [38]:
import networkx as nx
import matplotlib.pyplot as plt
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(df, "subject", "object", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [37]:
plt.figure(figsize=(12,12))

pos = nx.fruchterman_reingold_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
# edge_labels = nx.get_edge_attributes(G,'relation')
# for old_key in list(edge_labels):
#     new_key = old_key[:2]
#     edge_labels[new_key] = edge_labels.pop(old_key)
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.show()

KeyboardInterrupt: 

<Figure size 864x864 with 0 Axes>

In [39]:
G.number_of_nodes()

6101

In [40]:
G.size()

10000